In [22]:
!pip install qiskit --quiet
!pip install qiskit-aer --quiet
!pip install pylatexenc --quiet

In [23]:
import qiskit
print("qiskit version:", qiskit.__version__)

qiskit version: 2.1.0


<img src="https://raw.githubusercontent.com/Qiskit/qiskit-tutorials/master/images/qiskit-heading.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="500 px" align="left">

# Quantum Cryptography
This notebook consists in a possible implementation of the **BB84** cryptographic protocol on a quantum computer, reproducing **Quantum Key Distribution** and eavesdropper detection.

It makes use of IBM's QISKit, a python library that can manipulate quantum circuits, either via a simulation or a real execution on IBM's backend.



### Qiskit Package Versions

## Quantum Key Distribution
In 1984, building on the work of *Wiesner*, *Charles Bennett*, an IBM's researcher, and *Gilles Brassard*, of the Université de Montréal, developed the first **quantum cryptographic protocol**, which goes under the codename of **BB84**.

Suppose that *Alice* and *Bob* are connected via a **quantum channel** that they can use to exchange qubits. This channel is not used directly to send a private message, but only to exchange random qubits that after processing will compose the encryption key.  

If key sharing is completed successfully, this key can be used in the well known way as a **one-time pad** (OTP) to produce a safely encrypted message to be delivered over a **classical channel** using symmetrical cryptography. The key should be completely random, as long as the message, and discarded after use; the procedure can be repeated for every message that needs to be delivered.

More specifically *Alice* produces an **initial key**, selecting a sequence of **random bits**, '$0$' and '$1$', and picking a sequence of **polarization eigenstates**, with respect to a randomly chosen basis between: **rectilinear** $\{\lvert 0 \rangle,\  \lvert 1 \rangle\}$ and **diagonal** $\{\lvert \nearrow \rangle,\  \lvert \searrow \rangle\}$.

*Alice* encodes the classical bits of the key one by one in a **qubit**, by preparing each qubit in an eigenstate of the basis chosen, so that only by measuring the qubits in the **right basis** one can retrieve with **certainty** the right classical bit, just as it happens with quantum money. In the meantime *Alice* keeps a note (in a **table**) of the basis that she has picked for every single qubit she has encoded.

Now, using the quantum channel, she sends the stream of qubits to *Bob*, who is **unaware** of the basis used by *Alice* for the encoding. *Bob* receives these qubits prepared in a certain polarization eigenstate but, due to the **no-cloning theorem**, he is unable to recognize which basis *Alice* used, because he cannot distinguish **non-orthogonal states** with a single measurement. Nonetheless he proceeds anyway with measuring each photon's polarization using a basis chosen randomly (between rectilinear and diagonal), and he keeps a note of the measurement result and the associated basis that he used in a report **table**.

Statistically, *Bob* will pick the **right basis**, the same that *Alice* picked, about **$1/2$** of the times, and the wrong basis about **$1/2$** of the times. When he measures using the right basis he correctly retrieves the information bit of the key, but when he picks the wrong basis the information bit is not certain, since with respect to this basis, the qubit is in a **superposition** of the eigenstates of the right bases, and it can collapse in either two of them with equal probability of **$1/2.$**

For this reason *Alice* and *Bob* decide to **sift** their key, which in practical terms means that they discard from the key all the bits obtained via measurements made in the wrong basis, since they are not reliable. The price for this action is that the key will lose about **$1/2$** of its length, but the payoff is that they don't need to unveil their measurements, they just need to compare their tables, where they recorded the basis chosen, and they do that **after** the measurement has occurred.

So they open the **classical channel** and only now *Alice* tells (publicly) *Bob* which basis she used to encode the key; they **compare** the **tables** and discard the bits obtained measuring qubits in different basis. What they obtain is a perfectly correlate **sifted key**, the same for both of them, ready for use. This key can be employed as a one-time pad and once is used up completely, the procedure can be repeated again to produce a new random key.

What happens if we now introduce an **eavesdropper** in the communication? Suppose that *Eve* is able to intercept the qubits that Alice sends to Bob, and that she can also tap the classical communication channel. When she gets hold of the qubits she still doesn't know which basis *Alice* used, just like *Bob*. She is forced to make a guess, and she will pick the wrong basis **$1/2$** of the times. If she measures in the wrong basis she has **$1/2$** probability to make the qubit collapse in the wrong eigenstate, so that on the whole she will have altered about **$1/4$** of the original qubits. This is the main difference with classical crypto: thanks to quantum mechanics observing implies measuring, and if this is not done accordingly, it changes the actual state (key).

*Eve* produces a **candidate key** and passes on these (now altered) qubits to Bob who proceeds himself with his measurements. *Bob* constructs his table list of random basis and also obtains his candidate key, which will of course be different from *Eve*'s. When* Alice* broadcasts his basis table on the classical channel and *Bob* sift his  key accordingly, he will obtain a key different from *Alice*'s, unusable, since even in the same basis choice qubits will be different about **$1/4$** of the times. If *Alice* try to encrypt a message, symmetrical cryptography would fail and both *Alice* and *Bob* will know that communication has been compromised.

If *Alice* and *Bob* never compare their measurement and they only compare basis tables they have no way of knowing that the state has been altered, until the encrypted message is produced, sent and decryption fails. However they can decide to initiate **key sharing** by also comparing their measurement on a certain number of qubits, and, only when they are convinced that the channel is free of interference, they proceed with the actual key sharing. Of course the part of the key that represents the unveiled measurement has to be discarded from it. In real world application is comprises about $1/3$ of the whole key.

In this notebook I will be demonstrating exactly this behavior, how initial key sharing can be used to detect an eavesdropper.

## QKD proof of concept on a quantum  computer
Quantum Key Distribution requires special apparatus made for key sharing. Having at our disposal IBM's quantum computer, here we present a proof-of-concept of how the process can be realized, using real quantum measuring devices.

The key sharing part will be simulated using different quantum circuits one for each party (*Alice*, *Bob*, *Eve*) in the exchange, since we don't have a real quantum channel. We present first the simple case in which only *Alice* and *Bob* are present, and we later proceed to introduce *Eve* and demonstrate how she can be caught.

First we check for and import the required libraries:

In [24]:
# Import necessary libraries
import numpy as np
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.primitives import StatevectorSampler, BackendSamplerV2
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt


Next we do some preliminary settings to better manipulate quantum circuits and we set the number of available (qu)bits to 16

In [25]:
# Create registers
n = 16  # Maximum 23 for local backend (memory limitations)
qr = QuantumRegister(n, name='qr')
cr = ClassicalRegister(n, name='cr')

We create Alice's quantum circuit, made of $n$ qubits (and $n$ bits in a classical register, for measuring). We use $randint$ from numpy to generate a random number in the available range which will be our key and then we write the resulted number in binary and we memorize the key in a proper variable

In [26]:
# Alice's quantum circuit
alice = QuantumCircuit(qr, cr, name='Alice')

# Generate random key
alice_key = np.random.randint(0, high=2**n)
alice_key = np.binary_repr(alice_key, n)  # Convert to binary

Parse the generated key and we encode it in Alice's circuit, initializing her qubits to the computational basis: $\{\lvert 0 \rangle,\  \lvert 1 \rangle\}$, according to the value bit. Then we apply a rotation to about half of these qubits, so that about $1/2$ of them will now be in one of the eigenstates of the diagonal basis:  $\{\lvert \nearrow \rangle,\  \lvert \searrow \rangle\}$. We record the basis choice in a list (table) that will later be used for key verification.

In [27]:
# Encode key into qubits
for index, digit in enumerate(alice_key):
    if digit == '1':
        alice.x(qr[index])  # Flip to |1>

# Random basis selection
alice_table = []
for index in range(len(qr)):
    if np.random.random() > 0.5:  # 50% chance for diagonal basis
        alice.h(qr[index])
        alice_table.append('X')  # Diagonal basis
    else:
        alice_table.append('Z')  # Computational basis

How can we send this state to Bob? As said, we don't have another quantum computer, but we can create another quantum circuit, which we call $bob$, and initialize Bob's initial state to Alice's output state. To accomplish this task we define a helper function, *SendState*, that retrieves the qasm code of a given quantum circuit, $qc1$, does some filtering to extract the quantum gates applied, and produces new instructions that uses to initialize another circuit, $qc2$. This trick works because QISKit maintains a python dictionary of quantum circuits with their relative qasm instructions.

In [28]:
# get_qasm method needs the str label
# alternatively we can use circuits[0] but since dicts are not ordered
# it is not a good idea to put them in a func
# circuits = list(qp.get_circuit_names())

def send_state(qc1, qc2, qr):
    """Copy gates from qc1 to qc2 using qubit register qr"""
    for instr, qargs, cargs in qc1.data:
        gate_name = instr.name
        if gate_name == 'x':
            qc2.x(qargs[0])
        elif gate_name == 'h':
            qc2.h(qargs[0])
        elif gate_name == 'measure':
            continue  # Skip measurements
        else:
            raise Exception(f'Unsupported gate: {gate_name}')

Now we can create Bob's circuit and "send" Alice's qubits to Bob. We pretend that this state is unknown to Bob so that he doesn't know which basis to use and decides randomly that $1/2$ of the qubits are to be measured in the rectilinear basis and the other $1/2$ in the diagonal basis; we then record Bob's choice in his table list variable

In [29]:
# Bob's circuit
bob = QuantumCircuit(qr, cr, name='Bob')
send_state(alice, bob, qr)  # Transfer Alice's state

# Bob's random measurements
bob_table = []
for index in range(len(qr)):
    if np.random.random() > 0.5:  # Diagonal basis
        bob.h(qr[index])
        bob_table.append('X')
    else:
        bob_table.append('Z')
    bob.measure(qr[index], cr[index])  # Measure qubit

print("Bob's basis choices:", bob_table)

Bob's basis choices: ['Z', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'X', 'X']


/tmp/ipython-input-28-407392840.py:8: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 3.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  for instr, qargs, cargs in qc1.data:


Bob can now go ahead and measure all his qubits and store the measurement in the classical register. We build and run the circuit on the local backend, but, if a token is provided and enough credits are available, it can also be executed on the remote backend with 16 qubits, ibmqx5. Note that is very important that $shots=1$, since we have to pretend that Bob has only one measurement chance, otherwise he could statistically infer the basis used (you can try).

In [30]:
# Simulate quantum execution
simulator = AerSimulator()
compiled_circuit = transpile(bob, simulator)
result = simulator.run(compiled_circuit, shots=1).result()
counts = result.get_counts()
bob_results = list(counts.keys())[0][::-1]  # Extract results

In [31]:
# Compare basis choices
sifted_key = []
for i in range(n):
    if alice_table[i] == bob_table[i]:
        sifted_key.append(alice_key[i])

print("\nOriginal key:", alice_key)
print("Sifted key:", ''.join(sifted_key))
print(f"Key reduced by {100*(1-len(sifted_key)/n):.1f}% after sifting")


Original key: 0110101000010111
Sifted key: 01001
Key reduced by 68.8% after sifting


The histogram is not highly informative of course, but we can see that the measure has been performed correctly. Alice and Bob can switch over to the classical channel, compare their basis table lists, and discard qubits measured using different basis.

In [32]:
# Visualization
alice.draw(output='mpl', style='iqp')
plt.show()

plot_histogram(counts, title='Measurement Results')
plt.show()


We know that Bob will pick the wrong basis for $1/2$ of the qubits, so we can check that this theoretical probability is indeed replicated. We also know that although Bob picks the wrong basis, he can still end up with right eigenstate, and that he will do so about $1/2$ of the times, getting right $3/4$ of the qubits. We can check when Alice's and Bob's measurements coincide due to pure chance, although noting that this step is never performed in the actual key sharing step, but only in the inital sharing to test for eavesdropper.

In [33]:
n = 10

# Sample test keys (normally generated during BB84 simulation)
alice_key = ['0', '1', '0', '1', '0', '1', '0', '1', '0', '1']
bob_key   = ['0', '0', '0', '1', '1', '1', '0', '1', '1', '1']
keep = [0, 2, 3, 5, 6, 7]  # These are the positions where bases matched

# Count bit matches
acc = 0
for a_bit, b_bit in zip(alice_key, bob_key):
    if a_bit == b_bit:
        acc += 1

print('Percentage of qubits to be kept according to basis table comparison:', len(keep)/n)
print('Measurement convergence by random chance:', acc/n)

Percentage of qubits to be kept according to basis table comparison: 0.6
Measurement convergence by random chance: 0.7


Now before sifting the keys we perform a check on a certain number of the qubits, comparing their value to see if they have been altered. Since we have only 16 qubits, which is a really low number, we check all of them. Although the procedure is limited to exchange 16 qubits at a time it can be repeated as many times as needed.

In [34]:
new_alice_key = [alice_key[qubit] for qubit in keep]
new_bob_key = [bob_key[qubit] for qubit in keep]

acc = 0
for bit in zip(new_alice_key, new_bob_key):
    if bit[0] == bit[1]:
        acc += 1

print('Percentage of similarity between the keys: ', acc/len(new_alice_key))

Percentage of similarity between the keys:  1.0


If the qubits measured are the same can accept the new sifted keys. The new sifted keys are printed to stdout, of course this step is just to verify the rightness of the protocol, when the procedure is repeated, each party is not supposed to know the other's sifted key.

Note that, in the real world, quantum channel are subject to information loss since detectors are not perfectly efficient and some photons are going to be lost along the way. Thus, the similarity between the keys will hardly be $1.0$, but surely not as low as $0.75$ which we know is the case in which it has been eavesdropped. As a percentage cut-off we can pick $0.9$ and perform a check before calling the exchange successfull or invalid. You can try to insert a parameter that represents this loss as exercise.

In [35]:
if (acc//len(new_alice_key) == 1):
    print("Key exchange has been successfull")
    print("New Alice's key: ", new_alice_key)
    print("New Bob's key: ", new_bob_key)
else:
    print("Key exchange has been tampered! Check for eavesdropper or try again")
    print("New Alice's key is invalid: ", new_alice_key)
    print("New Bob's key is invalid: ", new_bob_key)

Key exchange has been successfull
New Alice's key:  ['0', '0', '1', '1', '0', '1']
New Bob's key:  ['0', '0', '1', '1', '0', '1']


Everything overlaps perfectly, that is indeed almost trivial. It's time to introduce Eve, the eavesdropper, and see what happens. We create Eve's circuit and we initiliaze it to Alice's state.

In [36]:
def SendState(sender: QuantumCircuit, receiver: QuantumCircuit, label=''):
    for instr, qargs, cargs in sender.data:
        receiver.append(instr, qargs, cargs)

eve = QuantumCircuit(qr, cr, name='Eve')
SendState(alice, eve, 'Alice')
qr = QuantumRegister(5)
cr = ClassicalRegister(5)

alice = QuantumCircuit(qr, cr, name='Alice')

# Alice prepares her qubits
for i in range(5):
    alice.h(i)
    alice.barrier()

# Now Eve receives the qubits
eve = QuantumCircuit(qr, cr, name='Eve')

# Simulate sending
SendState(alice, eve, 'Alice')

/tmp/ipython-input-36-3498409387.py:2: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 3.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  for instr, qargs, cargs in sender.data:


Just like Bob, Eve doesn't know which basis to use and she picks them randomly while recording her choice in a (table) list

In [37]:
eve_table = []
for index in range(len(qr)):
    if 0.5 < np.random.random():
        eve.h(qr[index])
        eve_table.append('X')
    else:
        eve_table.append('Z')

She measures according to her basis choice and she generates her candidate key

In [38]:

# Medir todos los qubits en Eve
for index in range(len(qr)):
    eve.measure(qr[index], cr[index])

# Crear simulador
simulator = AerSimulator()

# Transpilar circuito para el simulador
compiled_eve = transpile(eve, simulator)

# Ejecutar circuito
result = simulator.run(compiled_eve, shots=1).result()

# Obtener clave de Eve (como string binario)
eve_key = list(result.get_counts())[0]
eve_key = eve_key[::-1]  # invertir para que sea MSB→LSB (orden Qiskit)


Up to now, Eve did exactly what Bob did in the previous example. From this point on, however, things are a bit tricky. Eve has measured the state causing qubits to collapse in different eigenstates. This property is not easy to implement in QISKit because measurement results are stored in classical registered, while the qubits themselves are "unchanged". Therefore we need to update Eve's qubits to the new altered states starting from the results of the measures (Eve's key), reversing the instructions that Eve has executed, and apply them to qubits when necessary, which means when the basis choice was different.

You can try figure out yourself how a state is changed after a measurement, but remember that unitary operators in general don't commute.

In [39]:
# Update states to new eigenstates (of wrongly chosen basis)
for qubit, basis in enumerate(zip(alice_table, eve_table)):
    if basis[0] == basis[1]:
        print("Same choice for qubit: {}, basis: {}" .format(qubit, basis[0]))
    else:
        print("Different choice for qubit: {}, Alice has {}, Eve has {}" .format(qubit, basis[0], basis[1]))
        if eve_key[qubit] == alice_key[qubit]:
            eve.h(qr[qubit])
        else:
            if basis[0] == 'X' and basis[1] == 'Z':
                eve.h(qr[qubit])
                eve.x(qr[qubit])
            else:
                eve.x(qr[qubit])
                eve.h(qr[qubit])

Same choice for qubit: 0, basis: Z
Same choice for qubit: 1, basis: Z
Same choice for qubit: 2, basis: Z
Same choice for qubit: 3, basis: X
Same choice for qubit: 4, basis: X


Eve's altered state is now sent to Bob that performs the usual routine

In [40]:
# Qiskit 2.1.0 Compatible BB84 (Alice → Eve → Bob)
import numpy as np
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit_aer import AerSimulator  # Import AerSimulator from qiskit_aer
from qiskit.visualization import plot_histogram # Corrected import path
import matplotlib.pyplot as plt

# Crear registros compartidos
n_qubits = 5 # Set the number of qubits for the Eve scenario
qr = QuantumRegister(n_qubits, 'q')
cr = ClassicalRegister(n_qubits, 'c')

# Crear circuitos que comparten los registros
alice = QuantumCircuit(qr, cr, name='Alice')
eve   = QuantumCircuit(qr, cr, name='Eve')
bob   = QuantumCircuit(qr, cr, name='Bob')

# Alice elige bases aleatorias (Z o X) y bits (0 o 1)
alice_bases = []
alice_bits = []

for i in range(n_qubits):
    bit = int(np.random.rand() < 0.5)
    basis = 'X' if np.random.rand() < 0.5 else 'Z'
    alice_bits.append(bit)
    alice_bases.append(basis)

    if bit == 1:
        alice.x(qr[i])  # Codifica el bit

    if basis == 'X':
        alice.h(qr[i])  # Cambia a base X

# Generate alice_key based on alice_bits
alice_key = ''.join(map(str, alice_bits))

def SendState(sender, receiver):
    for gate in sender.data:
        receiver.append(gate.operation, gate.qubits, gate.clbits)

# Eve intercepta
SendState(alice, eve)

# Eve elige bases aleatorias y mide (simula ataque)
for i in range(n_qubits):
    if np.random.rand() < 0.5:
        eve.h(qr[i])
eve.measure(qr, cr)

# Luego borra mediciones para no duplicarlas en Bob
eve.data = [instr for instr in eve.data if instr.operation.name != 'measure']

# Eve reenvía a Bob
SendState(eve, bob)

# Bob mide en bases aleatorias
bob_bases = []
for i in range(n_qubits):
    basis = 'X' if np.random.rand() < 0.5 else 'Z'
    bob_bases.append(basis)
    if basis == 'X':
        bob.h(qr[i])
    bob.measure(qr[i], cr[i])

# Ejecutar simulación
simulator = AerSimulator() # Use AerSimulator
result = simulator.run(bob, shots=1).result() # Use the run method

# Mostrar resultados
counts = result.get_counts()
plot_histogram(counts)
plt.show()

In [41]:
bob_key = list(result.get_counts(bob))[0]
bob_key = bob_key[::-1]

After the measure Alice and Bob share the basis table lists and perform the usual checks

In [42]:
keep = []
discard = []
for qubit, basis in enumerate(zip(alice_bases, bob_bases)): # Use alice_bases and bob_bases from the current circuit
    if basis[0] == basis[1]:
        print("Same choice for qubit: {}, basis: {}" .format(qubit, basis[0]))
        keep.append(qubit)
    else:
        print("Different choice for qubit: {}, Alice has {}, Bob has {}" .format(qubit, basis[0], basis[1]))
        discard.append(qubit)

# Use the actual number of qubits from the current circuit
n_qubits = len(qr)

acc = 0
# Iterate over the keys generated in the current scenario
for a_bit, b_bit in zip(alice_key, bob_key):
    if a_bit == b_bit:
        acc += 1

print('\nPercentage of qubits to be discarded according to table comparison: ', len(keep)/n_qubits)
print('Measurement convergence by additional chance: ', acc/n_qubits)

new_alice_key = [alice_key[qubit] for qubit in keep]
new_bob_key = [bob_key[qubit] for qubit in keep]

acc = 0
for bit in zip(new_alice_key, new_bob_key):
    if bit[0] == bit[1]:
        acc += 1

print('\nPercentage of similarity between the keys: ', acc/len(new_alice_key) if len(new_alice_key) > 0 else 0) # Handle division by zero

if (acc == len(new_alice_key) and len(new_alice_key) > 0): # Check for empty key case
    print("\nKey exchange has been successfull")
    print("New Alice's key: ", new_alice_key)
    print("New Bob's key: ", new_bob_key)
else:
    print("\nKey exchange has been tampered! Check for eavesdropper or try again")
    print("New Alice's key is invalid: ", new_alice_key)
    print("New Bob's key is invalid: ", new_bob_key)

Same choice for qubit: 0, basis: X
Different choice for qubit: 1, Alice has Z, Bob has X
Different choice for qubit: 2, Alice has Z, Bob has X
Same choice for qubit: 3, basis: Z
Same choice for qubit: 4, basis: X

Percentage of qubits to be discarded according to table comparison:  0.6
Measurement convergence by additional chance:  0.6

Percentage of similarity between the keys:  0.6666666666666666

Key exchange has been tampered! Check for eavesdropper or try again
New Alice's key is invalid:  ['0', '1', '0']
New Bob's key is invalid:  ['1', '1', '0']


As you can see when Alice and Bob reveal their key to each other they notice a discordance: Eve has been caught!  To really get the percentages right, you can try repeating the experiment multiple times or you can write a higher routine that iterates the key sharing; in either case you will se that they converge to the expected values.